The tool agent goes off and runs on its own, and then returns the result to the original agent.

references: 
- https://openai.github.io/openai-agents-python/tools/#agents-as-tools

In [1]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
)
load_dotenv()

True

In [2]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


## define agents

In [3]:
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
    model=model_name,

)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
    model=model_name
)

orchestrator_agent = Agent(
    name="orchestrator_agent",
    model=model_name,
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
        "You only speak English. Do not translate the user's message yourself."
        "You can use the tools to translate the user's message."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

## execute agents

In [4]:
orchestrator_agent.model

'gpt-4o-mini'

In [5]:
try:
    with trace(workflow_name="translation_agent_as_tools", group_id="mdsi") as tr:
        result = await Runner.run(
            orchestrator_agent,
            "Greetings in French and Arabic",
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),

        )
except Exception as e:
    print(f"Error: {e}")


print(result.final_output)


The translation of "Greetings" is "Salut" in French and "Saludos" in Spanish. If you need the Arabic translation, please let me know!


In [6]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={tr.trace_id}"

print(f"Tracing URL: {tracing_url}")


Tracing URL: https://platform.openai.com/traces/trace?trace_id=trace_cec973ed3f334ea284ad338ff8ca2d99


## Visualise the Agent

In [8]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(orchestrator_agent, filename="viz/06_agent_as_tool.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
